### **Required Library/Dataset**

In [1]:
!git clone https://github.com/Song-Joo-Young/MobileNetV1-Optimization-for-CIFAR10

Cloning into 'MobileNetV1-Optimization-for-CIFAR10'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 115 (delta 35), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (115/115), 23.41 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [2]:
import os
os.chdir('MobileNetV1-Optimization-for-CIFAR10')

In [3]:
import sys
sys.path.append('/content/MobileNetV1-Optimization-for-CIFAR10/models')

In [4]:
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("Current GPU Index:", torch.cuda.current_device())
    print("Current GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Current GPU Index: 0
Current GPU Name: Tesla T4


In [5]:
# CIFAR-10 Dataset

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                 train=True,
                                 transform=transform,
                                 download=True)

test_dataset = datasets.CIFAR10(root='./cifar_10data/',
                                train=False,
                                transform=transforms.ToTensor())

100%|██████████| 170498071/170498071 [00:01<00:00, 98248806.85it/s] 


Extracting ./cifar_10data/cifar-10-python.tar.gz to ./cifar_10data/


In [6]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 393.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=58d7e581de4792eafc2bf448107ed05f7ad8223cc727a69f604afca6a8c55d1d
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=cd04ea2877a65d1a130d057926cc613f37c325a588c19d41108c5b579210ce51
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [7]:
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis

### **Baseline MobileNet**
* **Model**
  * from: https://github.com/kuangliu/pytorch-cifar/blob/master/models/mobilenet.py
* **Best accuracy** : [Test set] Average loss: 0.0056, Accuracy: 8426/10000 (84.26%)
* **FLOPS** : 48,412,672

* **hyperparameter**
  * lr=0.03
  * weight_decay=5e-4
  * batch size = 100
  * epoch = 100
  * training time : 3484.9894 sec

* **optimizer**
  * SGD

* **Data augmentation**
  *   
          transforms.Pad(4),
          transforms.RandomHorizontalFlip(),
          transforms.RandomCrop(32),
          transforms.ToTensor()

In [8]:
from models.Baseline_MobileNetV1 import MobileNet
model = MobileNet().to(device)

print("MobileNetV1 torchsummary")
summary(model, (3, 32, 32))

MobileNetV1 torchsummary
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
            Conv2d-3           [-1, 32, 32, 32]             288
       BatchNorm2d-4           [-1, 32, 32, 32]              64
            Conv2d-5           [-1, 64, 32, 32]           2,048
       BatchNorm2d-6           [-1, 64, 32, 32]             128
             Block-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 16, 16]             576
       BatchNorm2d-9           [-1, 64, 16, 16]             128
           Conv2d-10          [-1, 128, 16, 16]           8,192
      BatchNorm2d-11          [-1, 128, 16, 16]             256
            Block-12          [-1, 128, 16, 16]               0
           Conv2d-13          [-1, 128, 16, 16]           1,152
      BatchNor

In [9]:
inputs = torch.randn(1, 3, 32, 32).to(device)

flops = FlopCountAnalysis(model, inputs)
print('Total FLOPS :', flops.total())  # Total FLOPS

Total FLOPS : 48412672


In [10]:
loss_function = torch.nn.CrossEntropyLoss()
model = MobileNet().to(device)

model_path = '/content/MobileNetV1-Optimization-for-CIFAR10/best_weights/MobileNetV1_best_model_epoch_90.pkt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [11]:
model.eval()
test_loss, correct, total = 0, 0, 0

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
with torch.no_grad():  #using context manager
    for images, labels in test_loader :
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        test_loss += loss_function(output, labels).item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

        total += labels.size(0)

print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss /total, correct, total,
        100. * correct / total))

[Test set] Average loss: 0.0056, Accuracy: 8426/10000 (84.26%)



### **MobileNetV2**
* **Model**
  * MobileNetV2 from: https://github.com/chenhang98/mobileNet-v2_cifar10/blob/master
* **Best accuracy** : [Test set] Average loss: 0.0040, Accuracy: 8735/10000 (87.35%)
* **FLOPS** : 319,015,424


* **hyperparameter**
  * lr=0.1
  * weight_decay=4e-5
  * batch size = 100
  * momentum = 0.9
  * epoch = 25
  * training time : 3439.5674

* **optimizer**
  * SGD

* **Data augmentation**
  *   
          transforms.Pad(4),
          transforms.RandomHorizontalFlip(),
          transforms.RandomCrop(32),
          transforms.ToTensor()

In [12]:
from models.MobileNetV2 import MobileNetV2
model = MobileNetV2(10, alpha = 1).to(device)

print("MobileNetV2 torchsummary")
summary(model, (3, 32, 32))

MobileNetV2 torchsummary
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
            Conv2d-3           [-1, 32, 32, 32]           1,024
       BatchNorm2d-4           [-1, 32, 32, 32]              64
            Conv2d-5           [-1, 32, 32, 32]             288
       BatchNorm2d-6           [-1, 32, 32, 32]              64
            Conv2d-7           [-1, 16, 32, 32]             512
       BatchNorm2d-8           [-1, 16, 32, 32]              32
         BaseBlock-9           [-1, 16, 32, 32]               0
           Conv2d-10           [-1, 96, 32, 32]           1,536
      BatchNorm2d-11           [-1, 96, 32, 32]             192
           Conv2d-12           [-1, 96, 32, 32]             864
      BatchNorm2d-13           [-1, 96, 32, 32]             192
           Con

In [18]:
inputs = torch.randn(1, 3, 32, 32).to(device)

flops = FlopCountAnalysis(model, inputs)
print('Total FLOPS :', flops.total())  # Total FLOPS

Total FLOPS : 26681344


In [13]:
loss_function = torch.nn.CrossEntropyLoss()
model = MobileNetV2(10, alpha = 1).to(device)

model_path = '/content/MobileNetV1-Optimization-for-CIFAR10/best_weights/MobileNetV2_best_model_epoch_25.pkt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [14]:
model.eval()
test_loss, correct, total = 0, 0, 0

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
with torch.no_grad():  #using context manager
    for images, labels in test_loader :
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        test_loss += loss_function(output, labels).item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

        total += labels.size(0)

print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss /total, correct, total,
        100. * correct / total))

[Test set] Average loss: 0.0040, Accuracy: 8735/10000 (87.35%)



### **CustomMobileNet - Final version**
* **Model**
  * Architecture Adjustments
    * cfg = [32, 32, 32, (64,2), 64, 64, 64, (128,2), 128, 128, 128, 128, 128, 128, 128, 128, (256,2), 256, 256, 256, (512,2), 512]
    * Residual connection
  * Hyperparameter Tuning
  * Scheduler
  * Data Augmentation Techniques
    * Mix-up
  * Regularization Strategies
    * Drop out
    * Weight initialization
  
* **Best accuracy** :
  * Baseline : [Test set] Average loss: 0.0056, Accuracy: 8426/10000 (84.26%)
  * CustomMobileNet : [Test set] Average loss: 0.0032, Accuracy: 9185/10000 (91.85%) **[ +7.59% ]**

* **FLOPS** : 48,412,672 → 28,963,840 **[ -19,448,832 ]**
* **Total parameters** : 3,217,226 → 1,002,218 **[ -2,215,008 ]**

* **hyperparameter**
  * lr=(0.1), 0.94 * epoch
  * weight_decay=2.5e-3
  * momentum=0.9
  * batch size = 500
  * epoch = 95 + 10 = 105
  * training time : 3544.60

* **optimizer**
  * SGD

* **Data augmentation**
  * **mix-up(alpha = 1.0)**
  * Reference: Facebookreseach mixup-cifar10 (https://github.com/facebookresearch/mixup-cifar10)
  *   
          transforms.Pad(4),
          transforms.RandomHorizontalFlip(),
          transforms.RandomCrop(32),
          transforms.ToTensor()

In [15]:
from models.CustomMobileNet import CustomMobileNet
model = CustomMobileNet().to(device)

print("CustomMobileNet torchsummary")
summary(model, (3, 32, 32))

CustomMobileNet torchsummary
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 32, 32]             144
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 32, 32, 32]             512
       BatchNorm2d-6           [-1, 32, 32, 32]              64
            Conv2d-7           [-1, 32, 32, 32]             512
       BatchNorm2d-8           [-1, 32, 32, 32]              64
             Block-9           [-1, 32, 32, 32]               0
           Conv2d-10           [-1, 32, 32, 32]             288
      BatchNorm2d-11           [-1, 32, 32, 32]              64
           Conv2d-12           [-1, 32, 32, 32]           1,024
      BatchNorm2d-13           [-1, 32, 32, 32]              64
          

In [19]:
inputs = torch.randn(1, 3, 32, 32).to(device)

flops = FlopCountAnalysis(model, inputs)
print('Total FLOPS :', flops.total())  # Total FLOPS

Total FLOPS : 26681344


In [16]:
loss_function = torch.nn.CrossEntropyLoss()
model = CustomMobileNet().to(device)

model_path = '/content/MobileNetV1-Optimization-for-CIFAR10/best_weights/CustomMobileNet_best_model_epoch_95.pkt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [17]:
model.eval()
test_loss, correct, total = 0, 0, 0

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
with torch.no_grad():  #using context manager
    for images, labels in test_loader :
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        test_loss += loss_function(output, labels).item()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()

        total += labels.size(0)

print('[Test set] Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss /total, correct, total,
        100. * correct / total))

[Test set] Average loss: 0.0032, Accuracy: 9181/10000 (91.81%)

